# Linear Regression with PyTorch

Continuing where the previous Tutorial left off, we'll discuss one of the foundational algorithms of machine learning in this post: Linear regression. We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall and humidity (input variables or features) in a region. Here's the training data: 

![Drag Racing](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias:
    
    yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
    yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
    
Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity: 
    
![Graph](https://i.imgur.com/4DJ9f8X.png)    

The learning part of linear regression is to figure out a set of weights `w11, w12, ... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called gradient descent.

We begin by importing NumPy and PyTorch.

In [1]:
import numpy as np
import torch

# Training data

The training data can be represented using 2 matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]], dtype='float32')

In [3]:
# Targets (apples, oranges) 
targets = np.array([[56,70],
                   [81,101],
                   [119,133],
                   [22,37],
                   [103, 119]], dtype='float32')

We've seperated the input and target variables, because we'll operate on them seperately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then  convert them to PyTorch tensors as follows:

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)
print(inputs.shape)
print(targets.shape)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
torch.Size([5, 3])
torch.Size([5, 2])


## Linear regression model from scratch

The weights and biases(`w11,w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.5813, -0.1964, -0.0337],
        [-0.5410,  0.2549,  0.7979]], requires_grad=True)
tensor([1.6316, 0.0536], requires_grad=True)


`torch.randn` creates a tensor with the  given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

Our model is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![model](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows: 

In [6]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor. 

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-55.4104,  11.9474],
        [-70.7052,  24.3176],
        [-77.2131,  33.4177],
        [-67.3524, -14.6448],
        [-59.6897,  43.0453]], grad_fn=<AddBackward0>)


Let's compare the predictions of out model with the actual targets.

In [8]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see that there's a huge difference between the predictions of our model and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to just work. 

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method: 

- Calculate the difference between two matrices (`preds` and `targets`). 
- Square all the elements of the difference matrix to remoce negative values.
- Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean squared error (MSE).

In [9]:
# MSE loss 
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all of the lements in a tensor, and the `.numel` method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [10]:
# Compute loss 
loss = mse(preds, targets)
print(loss)

tensor(13598.1328, grad_fn=<DivBackward0>)


Here's how we can interpret the result: On average, each element in the prediction differs from the actual target by about 138 (square root of the loss 19044). And that's pretty bad, considering the numbers we are trying to predict are themselves in the range 50-200. Also, the result is called the loss, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [11]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t the weights matrix is itself a matrix, with the same dimensions.

In [12]:
# Gradients for weigths
print(w)
print(w.grad)

tensor([[-0.5813, -0.1964, -0.0337],
        [-0.5410,  0.2549,  0.7979]], requires_grad=True)
tensor([[-11869.6396, -13313.4941,  -8114.8916],
        [ -6077.6489,  -6698.7979,  -4081.4800]])


In [13]:
print(b)
print(b.grad)

tensor([1.6316, 0.0536], requires_grad=True)
tensor([-142.2742,  -72.3834])


The loss is a quadratic function of our weights and biases and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If a gradient element is positive: 

- increasing the element's value slightly will increase the loss.
- decreasing the element's value slightly will decrease the loss.

![graph](https://i.imgur.com/hFYoVgU.png)

If a gradient element is negative: 

- increasing the element's value slightly will decrease the loss.
- decreasing the element;s value slightly will increase the loss.

![graph](https://i.imgur.com/w3Wii7C.png)

The increase of decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling `.zero_()` method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lear to unexpected results.

In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t. the weights and biases
4. Adjust the weigths by subtracting a small quantity proporional to the gradient 
5. Reset the gradients to zero

Let's implement the above step by step

In [15]:
# Generate predictions 
preds = model(inputs)
print(preds)

tensor([[-55.4104,  11.9474],
        [-70.7052,  24.3176],
        [-77.2131,  33.4177],
        [-67.3524, -14.6448],
        [-59.6897,  43.0453]], grad_fn=<AddBackward0>)


In [16]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(13598.1328, grad_fn=<DivBackward0>)


In [17]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-11869.6396, -13313.4941,  -8114.8916],
        [ -6077.6489,  -6698.7979,  -4081.4800]])
tensor([-142.2742,  -72.3834])


Finally, we update the weight and biases the biases using the gradients computed above.

In [18]:
# Adjust weights & reset gradients 
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:
    
- We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weight and biases.
- We multiply the gradients with a really small number (`10^-5` in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm.
- After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.

In [19]:
print(w)
print(b)

tensor([[-0.4626, -0.0633,  0.0475],
        [-0.4802,  0.3218,  0.8387]], requires_grad=True)
tensor([1.6330, 0.0543], requires_grad=True)


With the new weights and biases, the model should have lower loss.

In [20]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(9205.1982, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases  slightly using gradient descent.

# Training for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [21]:
# Train for 100 epochs
for i in range(500):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward() 
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [22]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(2.1512, grad_fn=<DivBackward0>)


In [23]:
preds

tensor([[ 57.7579,  70.3049],
        [ 81.1866, 101.3217],
        [120.0658, 131.4834],
        [ 22.7925,  37.0028],
        [ 99.4696, 120.2896]], grad_fn=<AddBackward0>)

In [24]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables, and we can get even better results by training for a few more epochs.

## Linear regression using PyTorch built-ins

The model and training process above were implemented using basic matrix operations. But since this is such a common patter, PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [25]:
import torch.nn as nn

As before, we represent the inputs and targets and matrices

In [26]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64],[87, 134, 58],
                  [102, 43, 37], [69, 96, 70], [73, 67, 43],
                  [91, 88, 64], [87, 134, 58], [102, 43, 37],
                  [69, 96, 70], [73, 67, 43], [91, 88, 64],
                  [87, 137, 58],[102, 43, 37],[69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],[81, 101], [119, 133],
                  [22, 37], [103, 119], [56, 70],
                  [81, 101], [119, 133], [22, 37],
                  [103, 119], [56, 70], [81, 101],
                  [119, 133], [22, 37], [103, 119]],
                   dtype='float32')
                
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We are using 15 training examples this time, to illustrate how to work with large datasets in small batches.

# Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [27]:
from torch.utils.data import TensorDataset

In [28]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a prefedined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [29]:
from torch.utils.data import DataLoader

In [30]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

The data loader is typically used in a `for-in` loop. Let's look at an example. 

In [31]:
# Even though the data is shuffled, at the end all data is covered
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.]])
tensor([[103., 119.],
        [ 56.,  70.],
        [ 56.,  70.],
        [103., 119.],
        [119., 133.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [32]:
# Define model
# (3,2) --> 3 Inputs and 2 Outputs
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5619,  0.0780,  0.3154],
        [-0.2321,  0.5281,  0.3922]], requires_grad=True)
Parameter containing:
tensor([0.1179, 0.1150], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [33]:
# Parameters 
list(model.parameters())

[Parameter containing:
 tensor([[-0.5619,  0.0780,  0.3154],
         [-0.2321,  0.5281,  0.3922]], requires_grad=True),
 Parameter containing:
 tensor([0.1179, 0.1150], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [34]:
# Generate predictions 
preds = model(inputs)
preds

tensor([[-22.1170,  35.4235],
        [-23.9713,  50.5728],
        [-20.0291,  73.4428],
        [-42.1763,  13.6656],
        [ -9.0930,  62.2560],
        [-22.1170,  35.4235],
        [-23.9713,  50.5728],
        [-20.0291,  73.4428],
        [-42.1763,  13.6656],
        [ -9.0930,  62.2560],
        [-22.1170,  35.4235],
        [-23.9713,  50.5728],
        [-19.7952,  75.0272],
        [-42.1763,  13.6656],
        [ -9.0930,  62.2560]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the build-in loss function `mse_loss`.

In [35]:
# Import nn.functional
import torch.nn.functional as F

The nn.functional package contains many useful loss functions and several other utilities.

In [36]:
# Define loss function
loss_fn = F.mse_loss
print(type(inputs))
print(type(targets))
print(inputs.shape)
print(targets.shape)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([15, 3])
torch.Size([15, 2])


In [37]:
?nn.Linear

Let's compute the loss for the current predictions of our model.

In [38]:
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(6409.9917, grad_fn=<MseLossBackward>)


## Optimizer 

Instead of manually manipulating the models weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [39]:
# Define optimizer 
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD`, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent: 

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t. the weights and biases
4. Adjust the weights by subtracing a small quantity proportional to the gradient
5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs.

In [40]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for a given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients 
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress 
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:
- We use the data loader defined earlier to get batches of data for every iteration. 
- Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update, and `opt.zero_grad` to reset the gradients to zero. 
- We've also added a log statement which prints the loss from the last batch of data for every 10th epoch to track the progess of training. loss.item returns the actual values stored in the loss tensor.

Let's train the model for 100 epochs.

In [41]:
# Why is the loss jumping up again
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 35.6528
Epoch [20/100], Loss: 43.4497
Epoch [30/100], Loss: 23.3321
Epoch [40/100], Loss: 29.9068
Epoch [50/100], Loss: 16.2862
Epoch [60/100], Loss: 9.3505
Epoch [70/100], Loss: 14.5944
Epoch [80/100], Loss: 15.5268
Epoch [90/100], Loss: 8.9713
Epoch [100/100], Loss: 7.0942


Let's generate predictions using our model and verify that they're close to our targets.

In [42]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.5933,  70.6884],
        [ 82.3418,  98.6755],
        [116.9995, 135.4362],
        [ 23.9914,  40.0303],
        [100.5316, 114.0363],
        [ 57.5933,  70.6884],
        [ 82.3418,  98.6755],
        [116.9995, 135.4362],
        [ 23.9914,  40.0303],
        [100.5316, 114.0363],
        [ 57.5933,  70.6884],
        [ 82.3418,  98.6755],
        [119.4300, 138.0320],
        [ 23.9914,  40.0303],
        [100.5316, 114.0363]], grad_fn=<AddmmBackward>)

In [43]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Further Reading

We've covered a lot of ground in this tutorial, including linear regression and the gradient descent optimization algorithm. Here are a few resources if you'd like to dig deeper into these topics:

- To practice and test your skills, you can participate in the Boston Housing Price Prediction competition on Kaggle, a website that hosts data science competitions.

With this, we complete our discussion of linear regression in PyTorch, and we're ready to move on to the next topic: Logistic regression.